In [ ]:
!unzip /content/drive/MyDrive/ava.zip -d /content/ava1

In [ ]:
!mv /content/ava1/content/ava1/valid/complementary_colors/* /content/ava1/content/ava1/train/complementary_colors
!mv /content/ava1/content/ava1/valid/dutones/* /content/ava1/content/ava1/train/dutones
!mv /content/ava1/content/ava1/valid/hdr/* /content/ava1/content/ava1/train/hdr
!mv /content/ava1/content/ava1/valid/image_grain/* /content/ava1/content/ava1/train/image_grain
!mv /content/ava1/content/ava1/valid/light_on_white/* /content/ava1/content/ava1/train/light_on_white
!mv /content/ava1/content/ava1/valid/long_exposure/* /content/ava1/content/ava1/train/long_exposure
!mv /content/ava1/content/ava1/valid/macro/* /content/ava1/content/ava1/train/macro
!mv /content/ava1/content/ava1/valid/motion_blur/* /content/ava1/content/ava1/train/motion_blur
!mv /content/ava1/content/ava1/valid/negative_image/* /content/ava1/content/ava1/train/negative_image
!mv /content/ava1/content/ava1/valid/rule_of_thirds/* /content/ava1/content/ava1/train/rule_of_thirds
!mv /content/ava1/content/ava1/valid/shallow_dof/* /content/ava1/content/ava1/train/shallow_dof
!mv /content/ava1/content/ava1/valid/silhouttes/* /content/ava1/content/ava1/train/silhouttes
!mv /content/ava1/content/ava1/valid/soft_focus/* /content/ava1/content/ava1/train/soft_focus
!mv /content/ava1/content/ava1/valid/vanishing_point/* /content/ava1/content/ava1/train/vanishing_point

!mv /content/ava1/content/ava1/test/complementary_colors/* /content/ava1/content/ava1/train/complementary_colors
!mv /content/ava1/content/ava1/test/dutones/* /content/ava1/content/ava1/train/dutones
!mv /content/ava1/content/ava1/test/hdr/* /content/ava1/content/ava1/train/hdr
!mv /content/ava1/content/ava1/test/image_grain/* /content/ava1/content/ava1/train/image_grain
!mv /content/ava1/content/ava1/test/light_on_white/* /content/ava1/content/ava1/train/light_on_white
!mv /content/ava1/content/ava1/test/long_exposure/* /content/ava1/content/ava1/train/long_exposure
!mv /content/ava1/content/ava1/test/macro/* /content/ava1/content/ava1/train/macro
!mv /content/ava1/content/ava1/test/motion_blur/* /content/ava1/content/ava1/train/motion_blur
!mv /content/ava1/content/ava1/test/negative_image/* /content/ava1/content/ava1/train/negative_image
!mv /content/ava1/content/ava1/test/rule_of_thirds/* /content/ava1/content/ava1/train/rule_of_thirds
!mv /content/ava1/content/ava1/test/shallow_dof/* /content/ava1/content/ava1/train/shallow_dof
!mv /content/ava1/content/ava1/test/silhouttes/* /content/ava1/content/ava1/train/silhouttes
!mv /content/ava1/content/ava1/test/soft_focus/* /content/ava1/content/ava1/train/soft_focus
!mv /content/ava1/content/ava1/test/vanishing_point/* /content/ava1/content/ava1/train/vanishing_point

In [ ]:
import numpy as np 
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix, average_precision_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000

In [ ]:
### CONFIG ###
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

datasetFolderName='/content/ava1/content/ava1'
MODEL_FILENAME="model_cv.h5"
sourceFiles=[]
classLabels= [
              'complementary_colors', 
              'dutones', 
              'hdr', 
              'image_grain', 
              'light_on_white', 
              'long_exposure', 
              'macro', 
              'motion_blur', 
              'negative_image', 
              'rule_of_thirds',
              'shallow_dof',
              'silhouttes',
              'soft_focus',
              'vanishing_point'
]
batch_size = 16
epoch = 20

In [ ]:
def transferBetweenFolders(source, dest, splitRate):   
    global sourceFiles
    sourceFiles=os.listdir(source)
    if(len(sourceFiles)!=0):
        transferFileNumbers=int(len(sourceFiles)*splitRate)
        transferIndex=random.sample(range(0, len(sourceFiles)), transferFileNumbers)
        for eachIndex in transferIndex:
            shutil.move(source+str(sourceFiles[eachIndex]), dest+str(sourceFiles[eachIndex]))
    else:
        print("No file moved. Source empty!")

def transferAllClassBetweenFolders(source, dest, splitRate):
    for label in classLabels:
        transferBetweenFolders(datasetFolderName+'/'+source+'/'+label+'/', 
                               datasetFolderName+'/'+dest+'/'+label+'/', 
                               splitRate)

In [ ]:
### Lists to save images(X) and labels(Y)
X=[]
Y=[]

In [ ]:
def prepareNameWithLabels(folderName):
    sourceFiles=os.listdir(datasetFolderName+'/train/'+folderName)
    for val in sourceFiles:
        X.append(val)
        if (folderName==classLabels[0]):
            Y.append(0)
        elif (folderName==classLabels[1]):
            Y.append(1)
        elif (folderName==classLabels[2]):
            Y.append(2)
        elif (folderName==classLabels[3]):
            Y.append(3)
        elif (folderName==classLabels[4]):
            Y.append(4)
        elif (folderName==classLabels[5]):
            Y.append(5)
        elif (folderName==classLabels[6]):
            Y.append(6)
        elif (folderName==classLabels[7]):
            Y.append(7)
        elif (folderName==classLabels[8]):
            Y.append(8)
        elif (folderName==classLabels[9]):
            Y.append(9)
        elif (folderName==classLabels[10]):
            Y.append(10)
        elif (folderName==classLabels[11]):
            Y.append(11)
        elif (folderName==classLabels[12]):
            Y.append(12)
        elif (folderName==classLabels[13]):
            Y.append(13)

In [ ]:
# Organize file names and class labels in X and Y variables
prepareNameWithLabels(classLabels[0])
prepareNameWithLabels(classLabels[1])
prepareNameWithLabels(classLabels[2])       
prepareNameWithLabels(classLabels[3])       
prepareNameWithLabels(classLabels[4])       
prepareNameWithLabels(classLabels[5])       
prepareNameWithLabels(classLabels[6])       
prepareNameWithLabels(classLabels[7])       
prepareNameWithLabels(classLabels[8])       
prepareNameWithLabels(classLabels[9])       
prepareNameWithLabels(classLabels[10])       
prepareNameWithLabels(classLabels[11])       
prepareNameWithLabels(classLabels[12])       
prepareNameWithLabels(classLabels[13])       

X=np.asarray(X)
Y=np.asarray(Y)

In [ ]:
def getModel():
  densenet_model = tf.keras.applications.DenseNet201(include_top=False, input_shape=(256,256,3), weights='imagenet')
  model = Sequential()
  model.add(densenet_model)
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(units=264, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(units=264, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(units=14, activation='softmax'))

  model.compile(optimizer=Adam(learning_rate=0.0000065), loss='categorical_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(curve='PR', multi_label=True)])
  
  return model

In [ ]:
### Get mean average precision from predictions ###
def my_metrics(y_true, predictions):
    y_true_classes = label_binarize(y_true, [0,1,2,3,4,5,6,7,8,9,10,11,12,13])
    mAP = average_precision_score(y_true_classes, predictions)
    print("Mean Average Precision: {}".format(mAP))

In [ ]:
# Image dimensions
img_rows, img_cols =  256, 256

# Data paths
train_path=datasetFolderName+'/train/'
validation_path=datasetFolderName+'/validation/'
test_path=datasetFolderName+'/test/'

### Stratified K-Fold ###
skf = StratifiedKFold(n_splits=10, shuffle=True)
skf.get_n_splits(X, Y)
foldNum=0
for train_index, val_index in skf.split(X, Y):
    model=getModel()
    # Cut all images from validation to train (if any exists)
    transferAllClassBetweenFolders('validation', 'train', 1.0)
    foldNum+=1
    print("Results for fold",foldNum)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for eachIndex in range(len(X_val)):
        classLabel=''
        if (Y_val[eachIndex]==0):
            classLabel=classLabels[0]
        elif (Y_val[eachIndex]==1):
            classLabel=classLabels[1]
        elif (Y_val[eachIndex]==2):
            classLabel=classLabels[2]
        elif (Y_val[eachIndex]==3):
            classLabel=classLabels[3]
        elif (Y_val[eachIndex]==4):
            classLabel=classLabels[4]
        elif (Y_val[eachIndex]==5):
            classLabel=classLabels[5]
        elif (Y_val[eachIndex]==6):
            classLabel=classLabels[6]
        elif (Y_val[eachIndex]==7):
            classLabel=classLabels[7]
        elif (Y_val[eachIndex]==8):
            classLabel=classLabels[8]
        elif (Y_val[eachIndex]==9):
            classLabel=classLabels[9]
        elif (Y_val[eachIndex]==10):
            classLabel=classLabels[10]
        elif (Y_val[eachIndex]==11):
            classLabel=classLabels[11]
        elif (Y_val[eachIndex]==12):
            classLabel=classLabels[12]
        elif (Y_val[eachIndex]==13):
            classLabel=classLabels[13]

  
        # Copy the validation images to the validation folder
        shutil.move(datasetFolderName+'/train/'+classLabel+'/'+X_val[eachIndex], 
                    datasetFolderName+'/validation/'+classLabel+'/'+X_val[eachIndex])
        
    train_datagen = ImageDataGenerator(rescale=1.0/255.0)
    validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
        
    # Generate the batches
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
            validation_path,
            target_size=(img_rows, img_cols),
            batch_size=batch_size,
            class_mode='categorical',  # only data, no labels
            shuffle=False)   
   
    # Fit the model
    history=model.fit_generator(train_generator, 
                        epochs=epoch, validation_data=validation_generator)
    
    predictions = model.predict_generator(validation_generator, verbose=1)
    yPredictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    
    # Evaluate the model
    print("\n\n***Performance on Validation data***")    
    my_metrics(true_classes, predictions)
    print("\n\n")

74842112/74836368 [==============================] - 1s 0us/step
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
Results for fold 1
Found 10127 images belonging to 14 classes.
Found 1126 images belonging to 14 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
188/633 [=======>......................] - ETA: 1:40 - loss: 3.3121 - accuracy: 0.0876 - auc: 0.0785

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 211288080 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 131073 bytes but only got 10470. Skipping tag 582
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 196867 bytes but only got 10474. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 5. 
  warnings.warn(str(msg))


599/633 [===========================>..] - ETA: 7s - loss: 3.0152 - accuracy: 0.0998 - auc: 0.0809

...

556/633 [=========================>....] - ETA: 17s - loss: 3.0294 - accuracy: 0.0881 - auc_9: 0.0770

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 11046 bytes but only got 749. Skipping tag 700
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 9664 bytes but only got 749. Skipping tag 34377
  " Skipping tag %s" % (size, len(data), tag)


633/633 [==============================] - 177s 244ms/step - loss: 2.9977 - accuracy: 0.0904 - auc_9: 0.0775 - val_loss: 2.4769 - val_accuracy: 0.1938 - val_auc_9: 0.1718
Epoch 2/20
633/633 [==============================] - 151s 238ms/step - loss: 2.5199 - accuracy: 0.1631 - auc_9: 0.1189 - val_loss: 2.2364 - val_accuracy: 0.2924 - val_auc_9: 0.2581
Epoch 3/20
633/633 [==============================] - 151s 239ms/step - loss: 2.3386 - accuracy: 0.2313 - auc_9: 0.1850 - val_loss: 2.0135 - val_accuracy: 0.3787 - val_auc_9: 0.3517
Epoch 4/20
633/633 [==============================] - 151s 238ms/step - loss: 2.1566 - accuracy: 0.3012 - auc_9: 0.2518 - val_loss: 1.8628 - val_accuracy: 0.4133 - val_auc_9: 0.4170
Epoch 5/20
633/633 [==============================] - 151s 239ms/step - loss: 1.9711 - accuracy: 0.3585 - auc_9: 0.3129 - val_loss: 1.7382 - val_accuracy: 0.4764 - val_auc_9: 0.4645
Epoch 6/20
633/633 [==============================] - 151s 238ms/step - loss: 1.8461 - accuracy: 0.40

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


71/71 [==============================] - 13s 130ms/step


***Performance on Validation data***
Mean Average Precision: 0.5964066950174683



